**PART 1 - 初始化 关系模式R 和 函数依赖集合F**

    例如：
        * R = (ABCDEF)
        * F = {A → B, B → C}

In [2]:
R = ['A','B','C','D','E','F']

F = [[['A'],['B']],[['B'],['C','D']],[['D'],['E']],[['C','E'],['F']]]

**PART 2 - 计算 属性闭包集合{α+}**

    先算出单个属性的闭包，再遍历F算出所有属性闭包

In [3]:
def calculate_single_attribute_closure(original_alpha, F):
    """
    此函数用于计算单个属性的属性闭包
    ex:
    输入:['A'], F
    输出:['A','B','C','D','E','F'] 
    """
    alpha = original_alpha.copy()
    if alpha == []:
        return []
    result = alpha.copy()
    while True:
        for alpha_i, beta_i in F:
            #for α_i-->β_i, if α_i ⊂ result 
            if set(alpha_i).issubset(result):
                # result = result ∪ α_i
                result.extend([b for b in beta_i if b not in result])
        #if not changed after a loop, the calculation is finished 
        # 结果不再改变，说明属性闭包计算完成
        if result == alpha:
            break
        #if not finished, update alpha and continue calculations and comparisons 
        # 结果有所改变，继续计算和比较 
        alpha = result 
    return result

In [4]:
def calculate_attribute_closure_set( F ):
    """
    用来计算所有的属性闭包
    ex.
    输入:函数依赖集合F
    输出: 
    ['A']  -->  ['A', 'B', 'C', 'D', 'E', 'F']
    ['B']  -->  ['B', 'C', 'D', 'E', 'F']
    ['D']  -->  ['D', 'E']
    ['C', 'E']  -->  ['C', 'E', 'F']
    """
    #to store all attribute closures 用与存储所有的属性闭包
    attribute_closure_set = []
    for alpha, beta in F:
        result = calculate_single_attribute_closure(alpha, F)
        #to eliminate duplicate attribute closure 若属性闭包未重复，则添加
        if [alpha,sorted(result)] not in attribute_closure_set:
            attribute_closure_set.append([alpha,sorted(result)])
    return attribute_closure_set 

#calculate all attribute closures
attribute_closure_set = calculate_attribute_closure_set( F ) 

#print all attribute closures
for alpha, alpha_closure in attribute_closure_set:
    print(alpha,' --> ',alpha_closure)


['A']  -->  ['A', 'B', 'C', 'D', 'E', 'F']
['B']  -->  ['B', 'C', 'D', 'E', 'F']
['D']  -->  ['D', 'E']
['C', 'E']  -->  ['C', 'E', 'F']


**PART 3 - 计算 函数依赖闭包集合F+**

    方法一：算出属性闭包{α+}后即可得到函数依赖闭包F+的近似
    方法二：根据Armstrong's Axioms来得到完整的函数依赖闭包F+

In [5]:
F = [[['A'],['B']],[['B'],['C','D']],[['D'],['E']],[['C','E'],['F']]]

def calculate_function_dependency_closure_set_by_attribute_closure_set(F):
    """
    此函数用来计算函数依赖集合闭包F+ 
    """
    function_closure_set = []
    attribute_closure_set = calculate_attribute_closure_set(F)
    for alpha, beta in attribute_closure_set:
        for attribute in beta:
            function_closure_set.append([alpha, [attribute]])
    return function_closure_set

calculate_function_dependency_closure_set_by_attribute_closure_set(F)


[[['A'], ['A']],
 [['A'], ['B']],
 [['A'], ['C']],
 [['A'], ['D']],
 [['A'], ['E']],
 [['A'], ['F']],
 [['B'], ['B']],
 [['B'], ['C']],
 [['B'], ['D']],
 [['B'], ['E']],
 [['B'], ['F']],
 [['D'], ['D']],
 [['D'], ['E']],
 [['C', 'E'], ['C']],
 [['C', 'E'], ['E']],
 [['C', 'E'], ['F']]]

In [44]:
R = ['A','B','C']
F = [[['A'],['B']], [['B'],['C']]]

def get_subsets(s):
    """
    此函数用来计算集合的子集
    ex.
    输入:['A','B']
    输出:[[],['A'],['B'],['A','B']]
    #这个函数会使传入的s被修改, 需传入s.copy()
    """
    if not s:
        return [[]]
    x = s.pop()
    subsets = get_subsets(s)
    return subsets + [subset + [x] for subset in subsets]

def reflexive_rule(R,F):
    """  
    此函数运用Armstrong定理中的Reflexive rule来计算函数依赖闭包集合
    """
    new_F = F.copy()
    for attribute in R:
        if [[attribute],[attribute]] not in new_F:
            new_F.append([[attribute],[attribute]])
    return new_F

def augmentation_rule(R, F):
    """  
    此函数运用Armstrong定理中的Augmentation rule来计算函数依赖闭包集合
    """
    R_subsets = get_subsets(R.copy())
    R_subsets.remove([])
    new_F = F.copy()
    for alpha, beta in F:
        for subset in R_subsets:
            augmentation_alpha = sorted(list(set(subset) | set(alpha)))
            augmentation_beta = sorted(list(set(subset) | set(beta)))
            if [augmentation_alpha, augmentation_beta] not in new_F:
                new_F.append([augmentation_alpha, augmentation_beta])
    return new_F

def transitivity_rule(R, F):
    """  
    此函数运用Armstrong定理中的Transitivity rule来计算函数依赖闭包集合
    """
    new_F = F.copy()
    for alpha, beta in F:
        for alpha_i, beta_i in F:
            if beta == alpha_i and [alpha,beta_i] not in new_F:
                new_F.append([alpha,beta_i])
    return new_F

def calculate_function_dependency_closure_set_by_armstrong_axioms(R,F):
    """  
    此函数运用Armstrong定理来计算函数依赖闭包集合
    输入 R, F
    输出 F+
    """
    F = reflexive_rule(R, F)
    F = augmentation_rule(R, F)
    F_original = F.copy()
    while True:
        F = transitivity_rule(R,F)
        if F == F_original:
            break
        else:
            F_original = F
    for i, (alpha, beta) in enumerate(F):
        F[i] = [sorted(alpha), sorted(beta)]
    F.sort(key=lambda x: (len(x[0]), x[0], len(x[1]), x[1]))
    return F

calculate_function_dependency_closure_set_by_armstrong_axioms(R,F)



[[['A'], ['A']],
 [['A'], ['B']],
 [['A'], ['C']],
 [['A'], ['A', 'B']],
 [['A'], ['A', 'C']],
 [['A'], ['B', 'C']],
 [['A'], ['A', 'B', 'C']],
 [['B'], ['B']],
 [['B'], ['C']],
 [['B'], ['B', 'C']],
 [['C'], ['C']],
 [['A', 'B'], ['B']],
 [['A', 'B'], ['C']],
 [['A', 'B'], ['A', 'B']],
 [['A', 'B'], ['A', 'C']],
 [['A', 'B'], ['B', 'C']],
 [['A', 'B'], ['A', 'B', 'C']],
 [['A', 'C'], ['C']],
 [['A', 'C'], ['A', 'C']],
 [['A', 'C'], ['B', 'C']],
 [['A', 'C'], ['A', 'B', 'C']],
 [['B', 'C'], ['C']],
 [['B', 'C'], ['B', 'C']],
 [['A', 'B', 'C'], ['C']],
 [['A', 'B', 'C'], ['A', 'C']],
 [['A', 'B', 'C'], ['B', 'C']],
 [['A', 'B', 'C'], ['A', 'B', 'C']]]

**PART 4 - 计算 候选码Candidat key**

    候选码是最小的超码

In [6]:
def get_subsets(s):
    """
    此函数用来计算集合的子集
    ex.
    输入:['A','B']
    输出:[[],['A'],['B'],['A','B']]
    #这个函数会使传入的s被修改, 需传入s.copy()
    """
    if not s:
        return [[]]
    x = s.pop()
    subsets = get_subsets(s)
    return subsets + [subset + [x] for subset in subsets]

def get_proper_subsets(s):
    """
    此函数用来计算集合的真子集
    ex.
    输入:['A','B']
    输出:[[],['A'],['B']]
    """
    s_copy = s.copy()
    temp = s.copy()
    subsets = get_subsets(temp)
    subsets.remove(s_copy)
    return subsets

def determine_candidate_key(alpha,F):
    """
    此函数用来判断属性集合是否为候选码
    ex.
    输入:['A'],F
    输出:True
    输入:['B']
    输出:False
    输入:['A','B']
    输出:False(此为超码而不是候选码)
    """
    if(set(R).issubset(calculate_single_attribute_closure(alpha,F))):

        proper_subsets_of_alpha = get_proper_subsets(alpha)
        sorted_proper_subsets_of_alpha = sorted(proper_subsets_of_alpha, key=len, reverse=True)

        is_candidate_key = True
        for item in sorted_proper_subsets_of_alpha:
            result = calculate_single_attribute_closure(item,F)
            if set(R).issubset(result):
                is_candidate_key = False
                break
        return is_candidate_key
    return False

def select_candidate_key(F):
    """
    此函数用来算出候选码
    ex.
    输出:['A']
    """
    candidate_key_set = []
    for alpha, beta in F:
        temp = alpha.copy()
        if(determine_candidate_key(alpha,F)):
            candidate_key_set.append(temp)
    return candidate_key_set

candidate_key_set = select_candidate_key(F)

print(candidate_key_set)

[['A']]


**PART 5 - 计算 正则覆盖集Canonical cover set**

    对于 α → β，先合并α相同的依赖，再剔除α和β中的extraneous attribute(s)

In [7]:
def function_dependency_union(F):
    """
    此函数返回合并后的函数依赖集合
    ex.
    F 中有 A-->B, B-->C, A-->C
    返回 A-->(B,C) , B-->C
    即 [[['A'], ['B', 'C']], [['B'], ['C']]]
    """
    union_F = []
    for alpha, beta in F:
        union_beta = beta.copy()
        for alpha_i, beta_i in F:
            if alpha == alpha_i:
                union_beta.extend([b for b in beta_i if b not in union_beta])
        if [alpha,sorted(union_beta)] not in union_F:
            union_F.append([alpha,sorted(union_beta)])
    return union_F

F_temperary = [[['A'],['B']],[['B'],['C']],[['A'],['C']]]

print(function_dependency_union(F_temperary))


[[['A'], ['B', 'C']], [['B'], ['C']]]


In [8]:
union_F = [[['A'], ['B']], [['B'], ['C']], [['A','C'],['D']]]

def eliminate_left_extraneous(union_F):
    """ 
    此函数用来剔除依赖关系集合F中左侧的extraneous attribute
    """
    for i, (alpha, beta) in enumerate(union_F):
        replace = [alpha, beta]
        for subset in get_proper_subsets(alpha):
            if subset:
                new_closure = calculate_single_attribute_closure(subset, union_F)
                if set(beta).issubset(new_closure) and (len(subset) < len(replace[0])):
                    replace = [subset, beta]
        union_F[i] = replace
    return union_F

eli_left_union_F = eliminate_left_extraneous(union_F)
print(eli_left_union_F)

[[['A'], ['B']], [['B'], ['C']], [['A'], ['D']]]


In [9]:
union_F = [[['A'], ['B']], [['B'], ['C']], [['A'],['C','D']]]

def eliminate_right_extraneous(union_F):
    """ 
    此函数用来剔除依赖关系集合F中右侧的extraneous attribute
    """
    for i, (alpha, beta) in enumerate(union_F):
        replace = [alpha, beta]
        for subset in get_proper_subsets(beta):
            if subset:
                temp_union_F = union_F.copy()
                temp_union_F[i] = [alpha, subset]
                new_closure = calculate_single_attribute_closure(alpha, temp_union_F)
                if set(beta).issubset(new_closure) and (len(subset) < len(replace[1])):
                    replace = [alpha, subset]
        union_F[i] = replace
    return union_F

eli_right_union_F = eliminate_right_extraneous(union_F)
print(eli_right_union_F)

[[['A'], ['B']], [['B'], ['C']], [['A'], ['D']]]


In [10]:
F = [[['A'], ['B','C']], [['B'], ['C']], [['A'], ['B']], [['A','B'], ['C']]]

def compute_canonical_cover_set(F):
    """ 
    此函数用来获得依赖关系集合F的正则覆盖集
    ex.
    输入:F
    输出:F的正则覆盖集
    """
    F = function_dependency_union(F)
    F = eliminate_left_extraneous(F)
    F = function_dependency_union(F)
    F = eliminate_right_extraneous(F)
    return F

F = compute_canonical_cover_set(F)
print(F)

[[['A'], ['B']], [['B'], ['C']]]
